In [7]:
import pandas as pd


In [8]:
sheet_data = pd.read_excel(r"C:\Users\fsp_adolpho.salvador\Desktop\Konica Minolta\Desktop Cloud - Documentos\Desktop\Py\Pendencias\Arquivos\Mapa2.xlsx", sheet_name='Sheet1')

sheet_data.columns

Index([           'Unnamed: 0',              'Material',
                  'CE05_zva70',            'CE07_zva70',
                  'CE11_zva70',            'CE13_zva70',
                 'Denominação',            'Criado por',
                       'Canal',     'Pendências Gerais',
                  'CE01_zstok',            'CE03_zstok',
                  'CE05_zstok',            'CE07_zstok',
                  'CE09_zstok',            'CE11_zstok',
                  'CE13_zstok',         'Contr.qualid.',
           2025-01-22 00:00:00,     2025-02-05 00:00:00,
           2025-02-10 00:00:00,     2025-02-13 00:00:00,
           2025-02-27 00:00:00,     2025-03-03 00:00:00,
           2025-03-07 00:00:00,     2025-03-10 00:00:00,
           2025-03-20 00:00:00,     2025-03-21 00:00:00,
           2025-03-25 00:00:00,     2025-03-28 00:00:00,
           2025-04-02 00:00:00,     2025-04-16 00:00:00,
           2025-05-19 00:00:00,              'Montante',
       'Modelos Lista Técnica',

In [9]:
sheet_data['pendencia_total'] = sheet_data.filter(like='_zva70').sum(axis=1)

sheet_data['estoque_total'] = sheet_data.filter(like='_zstok').sum(axis=1)

sheet_data['FUP_total'] = sheet_data.filter(like='_FUP').sum(axis=1)

sheet_data['estoque_no_similar'] = sheet_data.apply(
    lambda row: row['Quant estoque'] - row['estoque_total'] if row['Quant estoque'] > 0 else 0,
    axis=1
)


In [10]:
sheet_data

,Unnamed: 0,Material,CE05_zva70,CE07_zva70,CE11_zva70,CE13_zva70,Denominação,Criado por,Canal,Pendências Gerais,...,Material.1,Similar1,Similar2,Similar3,Similar4,Quant estoque,pendencia_total,estoque_total,FUP_total,estoque_no_similar
0,0,20AL48470G,0,0,1,0,BOTÃO,ROBERTA MACHADO,VENDA INDIRETA,0,...,20AL48470G,20AL48470,-,-,-,89,1,16,0.0,73
1,1,25SA20150,0,0,1,0,COLAR DE POSICIONAMENTO,RODRIGO PEREIRA DOS ANJOS,VENDA DIRETA,0,...,0,0,0,0,0,0,1,16,0.0,0
2,2,40AA77660,0,0,4,0,ENGRENAGEM F 28 DENTES,REGILENE DE CASSIA LUIZ OLIVEIRA,VENDA DIRETA,1,...,40AA77660,-,-,-,-,3,4,1,0.0,2
3,3,4658015106,0,0,3,0,CONJUNTO DO ROLO,ROBERTA MACHADO,VENDA INDIRETA,0,...,4658015106,-,-,-,-,950,3,25,0.0,925
4,4,55FU20140,0,0,1,0,MOLA DE FIXAÇÃO,RODRIGO PEREIRA DOS ANJOS,VENDA DIRETA,0,...,0,0,0,0,0,0,1,18,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,172,AC4J668800,0,0,2,0,Protection Sheet /6,JESSIE OLIVEIRA SILVA,VENDA DIRETA,0,...,0,0,0,0,0,0,2,0,0.0,0
173,173,AC57R70111,0,0,4,0,UNIDADE DE CARGA,ROBERTA MACHADO,VENDA INDIRETA,3,...,0,0,0,0,0,0,4,13,0.0,0
174,174,AC57R70122,0,0,1,9,UNIDADE DE CARGA,PATRICIA ALVES PELAKOSKI/PATRICIA ALVES PELAKO...,VENDA INDIRETA/VENDA INDIRETA/VENDA INDIRETA,26,...,AC57R70122,AC57R70111,AC57R70100,-,-,2849,10,91,0.0,2758
175,175,ACDN0Y1,0,0,21,0,FRASCO COLETOR DE TONER RESIDUAL,ROBERTA MACHADO/ROBERTA MACHADO/EDITH PEREIRA ...,VENDA INDIRETA/VENDA INDIRETA/VENDA DIRETA,0,...,ACDN0Y1,ACDNWY1,-,-,-,6889,21,193,0.0,6696


In [11]:
sheet_data['projecao_compra'] = sheet_data['pendencia_total'] - (sheet_data['estoque_total'] + sheet_data['FUP_total'] + sheet_data['estoque_no_similar'])
sheet_data['projecao_compra'] = sheet_data['projecao_compra'].apply(lambda x: max(x, 0))

In [12]:
sheet_data

,Unnamed: 0,Material,CE05_zva70,CE07_zva70,CE11_zva70,CE13_zva70,Denominação,Criado por,Canal,Pendências Gerais,...,Similar1,Similar2,Similar3,Similar4,Quant estoque,pendencia_total,estoque_total,FUP_total,estoque_no_similar,projecao_compra
0,0,20AL48470G,0,0,1,0,BOTÃO,ROBERTA MACHADO,VENDA INDIRETA,0,...,20AL48470,-,-,-,89,1,16,0.0,73,0.0
1,1,25SA20150,0,0,1,0,COLAR DE POSICIONAMENTO,RODRIGO PEREIRA DOS ANJOS,VENDA DIRETA,0,...,0,0,0,0,0,1,16,0.0,0,0.0
2,2,40AA77660,0,0,4,0,ENGRENAGEM F 28 DENTES,REGILENE DE CASSIA LUIZ OLIVEIRA,VENDA DIRETA,1,...,-,-,-,-,3,4,1,0.0,2,1.0
3,3,4658015106,0,0,3,0,CONJUNTO DO ROLO,ROBERTA MACHADO,VENDA INDIRETA,0,...,-,-,-,-,950,3,25,0.0,925,0.0
4,4,55FU20140,0,0,1,0,MOLA DE FIXAÇÃO,RODRIGO PEREIRA DOS ANJOS,VENDA DIRETA,0,...,0,0,0,0,0,1,18,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,172,AC4J668800,0,0,2,0,Protection Sheet /6,JESSIE OLIVEIRA SILVA,VENDA DIRETA,0,...,0,0,0,0,0,2,0,0.0,0,2.0
173,173,AC57R70111,0,0,4,0,UNIDADE DE CARGA,ROBERTA MACHADO,VENDA INDIRETA,3,...,0,0,0,0,0,4,13,0.0,0,0.0
174,174,AC57R70122,0,0,1,9,UNIDADE DE CARGA,PATRICIA ALVES PELAKOSKI/PATRICIA ALVES PELAKO...,VENDA INDIRETA/VENDA INDIRETA/VENDA INDIRETA,26,...,AC57R70111,AC57R70100,-,-,2849,10,91,0.0,2758,0.0
175,175,ACDN0Y1,0,0,21,0,FRASCO COLETOR DE TONER RESIDUAL,ROBERTA MACHADO/ROBERTA MACHADO/EDITH PEREIRA ...,VENDA INDIRETA/VENDA INDIRETA/VENDA DIRETA,0,...,ACDNWY1,-,-,-,6889,21,193,0.0,6696,0.0


In [13]:
valores_unicos = sheet_data['estoque_no_similar'].unique()
valores_unicos

array([   73,     0,     2,   925,  1791,   637,    30,  3388,   756,
         107,    12,    19,   139,  1171,  1146,  1490,  1006,   660,
          17,   114,   589,  2636,   338,   658,  1093,   647,   256,
         236,   224,   707,  2245, 12805,   480,   715,  2553,   562,
        1817,   308,  1157,   783,     9,  2025,   753,    16,    25,
          96,  4710,   388,  4033,  6804,   238,   157,  2068,    87,
        2084,   822,   109,     8,    32,  1682,   205,  4209,   190,
          14,   135,   129,  1098,  2758,  6696,     6], dtype=int64)

In [14]:

caminho_saida = r'C:\Users\fsp_adolpho.salvador\Desktop\Konica Minolta\Desktop Cloud - Documentos\Desktop\Py\Analise\Pendencia\Mapa Analisado1.0.xlsx'

sheet_data.to_excel(caminho_saida, index=True)

In [15]:
def atender_estoque_ajustado_v2(row):
    if row['projecao_compra'] == 0:
        for col in ['CE05_zstok', 'CE13_zstok', 'CE11_zstok', 'CE09_zstok', 'CE07_zstok', 'CE01_zstok', 'estoque_total', 'estoque_no_similar']:
            if row[col] >= row['pendencia_total']:
                if col == 'estoque_no_similar':
                    return "Atender Pelo Similar"
                else:
                    return f"Atender pelo {col.split('_')[0]}"
        if row['FUP_total'] >= row['pendencia_total']:
            return "Aguardar IMP"
        else:
            return "Sem estoque disponível"
    else:
        return row['projecao_compra']  

sheet_data['priorizacao_estoque'] = sheet_data.apply(atender_estoque_ajustado_v2, axis=1)



In [16]:
sheet_data

,Unnamed: 0,Material,CE05_zva70,CE07_zva70,CE11_zva70,CE13_zva70,Denominação,Criado por,Canal,Pendências Gerais,...,Similar2,Similar3,Similar4,Quant estoque,pendencia_total,estoque_total,FUP_total,estoque_no_similar,projecao_compra,priorizacao_estoque
0,0,20AL48470G,0,0,1,0,BOTÃO,ROBERTA MACHADO,VENDA INDIRETA,0,...,-,-,-,89,1,16,0.0,73,0.0,Atender pelo CE13
1,1,25SA20150,0,0,1,0,COLAR DE POSICIONAMENTO,RODRIGO PEREIRA DOS ANJOS,VENDA DIRETA,0,...,0,0,0,0,1,16,0.0,0,0.0,Atender pelo CE11
2,2,40AA77660,0,0,4,0,ENGRENAGEM F 28 DENTES,REGILENE DE CASSIA LUIZ OLIVEIRA,VENDA DIRETA,1,...,-,-,-,3,4,1,0.0,2,1.0,1.0
3,3,4658015106,0,0,3,0,CONJUNTO DO ROLO,ROBERTA MACHADO,VENDA INDIRETA,0,...,-,-,-,950,3,25,0.0,925,0.0,Atender pelo CE11
4,4,55FU20140,0,0,1,0,MOLA DE FIXAÇÃO,RODRIGO PEREIRA DOS ANJOS,VENDA DIRETA,0,...,0,0,0,0,1,18,0.0,0,0.0,Atender pelo CE11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,172,AC4J668800,0,0,2,0,Protection Sheet /6,JESSIE OLIVEIRA SILVA,VENDA DIRETA,0,...,0,0,0,0,2,0,0.0,0,2.0,2.0
173,173,AC57R70111,0,0,4,0,UNIDADE DE CARGA,ROBERTA MACHADO,VENDA INDIRETA,3,...,0,0,0,0,4,13,0.0,0,0.0,Atender pelo CE09
174,174,AC57R70122,0,0,1,9,UNIDADE DE CARGA,PATRICIA ALVES PELAKOSKI/PATRICIA ALVES PELAKO...,VENDA INDIRETA/VENDA INDIRETA/VENDA INDIRETA,26,...,AC57R70100,-,-,2849,10,91,0.0,2758,0.0,Atender pelo CE13
175,175,ACDN0Y1,0,0,21,0,FRASCO COLETOR DE TONER RESIDUAL,ROBERTA MACHADO/ROBERTA MACHADO/EDITH PEREIRA ...,VENDA INDIRETA/VENDA INDIRETA/VENDA DIRETA,0,...,-,-,-,6889,21,193,0.0,6696,0.0,Atender pelo CE13


In [17]:
caminho_saida = r'C:\Users\fsp_adolpho.salvador\Desktop\Konica Minolta\Desktop Cloud - Documentos\Desktop\Py\Analise\Pendencia\Arquivos\Mapa Analisado2.0.xlsx'

sheet_data.to_excel(caminho_saida, index=True)